In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

import pandas as pd


spark  = SparkSession.builder \
    .appName("Data Analysis") \
    .config("spark.executor.memory", "8g")\
    .config("spark.driver.memory", "8g")\
    .config("spark.executor.cores", "4")\
    .config("spark.executor.instances","8")\
    .config("spark.sql.shuffle.partitions","96")\
    .config("spark.default.parallelism","96")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/05 13:49:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100)
spark.conf.set("spark.sql.repl.eagerEval.maxNumRows", 300)

spark.conf.set("spark.sql.debug.maxToStringFields", 100)

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_rows", None)

In [3]:
base_final_gest_limp = (spark
            .read
            .option('nullValue', 'NA')
            .csv('/data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/Karine/Banco/basefinal_gest_limp.csv', header=True))

In [4]:
base_final_gest_limp.head()

Row(apgar1_sinasc='9', apgar5_sinasc='10', cod_raca_cor_pessoa_eq='4', codanomal_sinasc=None, codmunres_sinasc='292810', consprenat_sinasc='10', consultas_sinasc='4', dt_notific_mae=None, dtnasc_sinasc='2014-06-09', dtnascmae_sinasc='1985-01-18', dtobito_sim.y=None, escmae_sinasc='4', estcivmae_sinasc='2', gravidez_sinasc='1', id_agravo_mae=None, id_cidacs_mae_sinasc='849297898', id_cidacs_sinasc_v5='448674125', idademae_sinasc='29', idanomal_sinasc='2', mesprenat_sinasc='2', parto_sinasc='2', peso_sinasc='3770', qtdfilmort_sinasc='0', qtdfilvivo_sinasc='1', racacormae_sinasc='4', semagestac_sinasc='40', sexo_sinasc='2', tpapresent_sinasc='1', tpconfirma_mae=None, tpesquema_mae=None, tpevidenci_mae=None, tpteste1_mae=None, tra_dt_sc=None, marc_pbf_eq_v4='1', id_cidacs_pop100_v4='1205803674', ind_mae_sinasc_cidacs='0', dtultmenst_sinasc='2013-08-29', qtdgestant_sinasc='1')

In [5]:
base_final_gest_limp.columns

['apgar1_sinasc',
 'apgar5_sinasc',
 'cod_raca_cor_pessoa_eq',
 'codanomal_sinasc',
 'codmunres_sinasc',
 'consprenat_sinasc',
 'consultas_sinasc',
 'dt_notific_mae',
 'dtnasc_sinasc',
 'dtnascmae_sinasc',
 'dtobito_sim.y',
 'escmae_sinasc',
 'estcivmae_sinasc',
 'gravidez_sinasc',
 'id_agravo_mae',
 'id_cidacs_mae_sinasc',
 'id_cidacs_sinasc_v5',
 'idademae_sinasc',
 'idanomal_sinasc',
 'mesprenat_sinasc',
 'parto_sinasc',
 'peso_sinasc',
 'qtdfilmort_sinasc',
 'qtdfilvivo_sinasc',
 'racacormae_sinasc',
 'semagestac_sinasc',
 'sexo_sinasc',
 'tpapresent_sinasc',
 'tpconfirma_mae',
 'tpesquema_mae',
 'tpevidenci_mae',
 'tpteste1_mae',
 'tra_dt_sc',
 'marc_pbf_eq_v4',
 'id_cidacs_pop100_v4',
 'ind_mae_sinasc_cidacs',
 'dtultmenst_sinasc',
 'qtdgestant_sinasc']

In [6]:
number_of_variables = len(base_final_gest_limp.columns)
number_of_variables

38

In [7]:
base_final_gest_limp = base_final_gest_limp.withColumnRenamed('dtobito_sim.y','dtobito_sim')

In [8]:
date_variables = [] 

for variable in base_final_gest_limp.columns: 
    if str(variable).lower().startswith('dt'): 
        date_variables.append(variable)

In [9]:
date_variables

['dt_notific_mae',
 'dtnasc_sinasc',
 'dtnascmae_sinasc',
 'dtobito_sim',
 'dtultmenst_sinasc']

## **Análise das datas**

In [10]:
for variable in date_variables:
    print('variável: {variable}')
    base_final_gest_limp.select(F.min(variable)).show()
    base_final_gest_limp.select(F.max(variable)).show()
    print('------------------------------------')

variável: {variable}


+-------------------+
|min(dt_notific_mae)|
+-------------------+
|         2011-01-01|
+-------------------+



+-------------------+
|max(dt_notific_mae)|
+-------------------+
|         2020-12-31|
+-------------------+

------------------------------------
variável: {variable}


+------------------+
|min(dtnasc_sinasc)|
+------------------+
|        2011-01-01|
+------------------+



+------------------+
|max(dtnasc_sinasc)|
+------------------+
|        2020-12-31|
+------------------+

------------------------------------
variável: {variable}


...

+---------------------+
|max(dtnascmae_sinasc)|
+---------------------+
|           2009-10-05|
+---------------------+

------------------------------------
variável: {variable}


+----------------+
|min(dtobito_sim)|
+----------------+
|      2011-01-01|
+----------------+



+----------------+
|max(dtobito_sim)|
+----------------+
|      2020-12-31|
+----------------+

------------------------------------
variável: {variable}


+----------------------+
|min(dtultmenst_sinasc)|
+----------------------+
|            1931-02-11|
+----------------------+



+----------------------+
|max(dtultmenst_sinasc)|
+----------------------+
|            2020-12-27|
+----------------------+

------------------------------------


## **Análise da variável racacormae_sinasc**

In [13]:
spark.sql("SELECT COUNT({df.racacormae_sinasc}) FROM {df} WHERE {df.racacormae_sinasc} != 'NA'", df=base_final_gest_limp).show()

+------------------------+
|count(racacormae_sinasc)|
+------------------------+
|                16609493|
+------------------------+

